<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/text_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

def fetch_urls(base_url, start_page, end_page):
    links = []

    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:108.0) Gecko/20100101 Firefox/108.0"}

    for page in range(start_page, end_page + 1):
        url = f"{base_url}/page/{page}/" if page < 1 else base_url
        print(f"Fetching: {url}")
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            container = soup.find('div', id='main-content')
            if container:
                h2_tags = container.find_all('h2')
                for h2 in h2_tags:
                    a_tag = h2.find('a')
                    if a_tag and a_tag.has_attr('href'):
                        links.append(a_tag['href'])
        else:
            print(f"Failed to fetch {url}: Status code {response.status_code}")

        sleep(2)

    return links

base_url = "https://alanwatts.org/category/transcripts"
start_page = 1
end_page = 13
all_links = fetch_urls(base_url, start_page, end_page)
print(all_links)

In [ ]:
json_objects = []

In [ ]:
def fetch_and_process(url):
  headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:108.0) Gecko/20100101 Firefox/108.0"}
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    transcript_div = soup.find('div', class_='ts-transcript')
    if transcript_div:
      paragraphs = transcript_div.find_all('p')
      for p in paragraphs:
        text = p.get_text(strip=True)
        if len(text) > 100:
          json_obj = {
              "instruction" : "",
              "input" : "",
              "output" : text
          }
          json_objects.append(json_obj)
    else:
      print(f'no transcript class: {url}')
  else:
    print('failed to fetch url')

for link in list(set(all_links)):
  fetch_and_process(link)
  sleep(2)

print(json.dumps(json_objects, indent=4))

In [ ]:
from google.colab import files
import io

with open('data.json', 'w') as file:
  json.dump(json_objects, file, indent=4, ensure_ascii=False)

files.download('data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>